### automated browsing and regular expressions
<p>target website: www.nboard.dk </p>

browse the site for potential board members.do this automatically with `Selenium`

In [19]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

In [20]:
url = 'https://nboard.dk/search'
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get(url)

C:\Users\dell\AppData\Local\Temp/ipykernel_18756/315452120.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


In [21]:
from selenium.webdriver.common.by import By
cookie = driver.find_element(By.CSS_SELECTOR, '.cc-dismiss') #Here we use a CSS selector
cookie.click()

In [25]:
boardmember = driver.find_element("id","mat-checkbox-2") #Here we use the id attribute to find the boardmember box
boardmember.click()

In [28]:
import time

for i in range(5): #We scroll down 5 times and sleep for 3 seconds each time to wait for the webpage to load
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);") #Execute JavaScript on the browser that scroll down page
    time.sleep(3)

In [30]:
# Find the place to type in search text
candidate = driver.find_element("id","mat-input-4")
candidate.click() #And click
# Type the search text
candidate.send_keys('Hansen') #Use the `.send_keys` to type text. `.send_keys` imitates your computer keyboard

In [34]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(driver.page_source, 'lxml') #The Selenium Driver keeps the HTML in the `.page_source` property

# Find the place where the number of profiles is shown
results = soup.find_all('span', class_ = "ng-tns-c10-0")
print(results)
results = results[4] #More elements matches the search, so we take the number of profiles element only 结果数量存储在[4]，我也不知道为什么
print()
print(results)

# We only want the text content of the HTML
profiles = results.text #We take the text
profiles

[<span _ngcontent-c10="" class="ng-tns-c10-0" style="font-size: 15px;">Minimum bestyrelseserfaring: 0 år</span>, <span _ngcontent-c10="" class="ng-tns-c10-0" style="font-size: 12px; margin-bottom: 20px;">Vælg din virksomhed nedenfor, så finder vi kandidater med erfaring fra samme branche</span>, <span _ngcontent-c10="" class="ng-tns-c10-0" style="font-size: 15px;">Minimum bestyrelseserfaring: 0 år</span>, <span _ngcontent-c10="" class="ng-tns-c10-0" style="margin-right: 10px;">Begrænset adgang - <a _ngcontent-c10="" class="ng-tns-c10-0" href="/authentication/signup" routerlink="/authentication/signup">Opret en <b _ngcontent-c10="" class="ng-tns-c10-0">gratis</b> bruger</a> eller <a _ngcontent-c10="" class="ng-tns-c10-0" href="/authentication/login" routerlink="/authentication/login">log ind</a> for at få fuld adgang</span>, <span _ngcontent-c10="" class="ng-tns-c10-0"><b _ngcontent-c10="" class="ng-tns-c10-0">Antal resultater:</b> 105 | <b _ngcontent-c10="" class="ng-tns-c10-0">Sorteri

'Antal resultater: 105 | Sortering: Mest relevante først.'

In [45]:
import re
number_profiles = re.search(r'\d+', profiles)
'''
The '\d' searches for digits in the string, and the '+' tells regex to search for all digits. 
The 'r' in the front of the string makes the string into a raw string; 
it means that for example \n (new line) is not interpreted as new line, but is just seen as '\n'.
'''
print(number_profiles)
print(type(number_profiles))
number_profiles = number_profiles.group()
'''
group方法返回一个或者多个匹配的子组。 如果只有一个参数，结果就是一个字符串，如果有多个参数，
结果就是一个元组（每个参数对应一个项），如果没有参数，整个匹配都被返回。 如果一个参数值为0，即是group. 

# 不知道为什么返回的是105
'''
print(number_profiles)

<re.Match object; span=(18, 21), match='105'>
<class 're.Match'>
105


## Regex - regular expression
### See more in this [tutorial](https://www.digitalocean.com/community/tutorials/an-introduction-to-regex-in-python) and this [guide](https://www.regular-expressions.info/refquick.html)
* \+ = 1 or more times  -- e.g. "a+" will match: "a", and "aaa"
* \* = 0 or more times  -- e.g. "ba*" will match: "b", and "ba", and "baaa"
* {3} = exactly three times --- e.g. "ba{3}" will match "baaa", but not "baa"
* ? = once or none
* \\ = escape character, used to find characters that has special meaning with regex: e.g. \+ \*
* [] = allows you to define a set of characters
* () = groups a part of the regular expression
* ^ = applied within a set, it becomes the inverse of the set defined. Applied outside a set it entails the beginning of a string. $ entails the end of a string.
* . = any characters except line break
* | = or statement. -- e.g. a|b means find characters a or b.
* \d = digits
* \D = non-digits.
* \s = whitespace-separator
* \w = matches alphanumeric character [a-zA-Z0-9_]
* \W = matches any non-alphanumeric character [^a-zA-Z0-9]

Sequences
* (?:) = Defines a Non-capturing group. -- e.g. "(?:abc)+", will match "abc" and "abcabcabc", but not "aabbcc"
* (?=)	= Positive lookahead - only match a certain pattern if a certain pattern comes after it.
* (?!)	= Negative lookahead - only match a certain pattern if **not** a certain pattern comes after it.
* (?<=)	= Positive lookbehind - only match a certain pattern if a certain pattern precedes it.
* (?<!) = Negative lookbehind - only match a certain pattern if **not** a certain pattern precedes it.

In [47]:
text = 'Gazprom halverer gasleverancerne til Europa via Nord Stream 1. Årsagen er ifølge selskabet vedligehold af en gasturbine. Den daglige gasforsyning via gasledningen vil fra onsdag morgen blive reduceret til 33 millioner kubikmeter, oplyser Gazprom.Det svarer til cirka 20 procent af den maksimale kapacitet, og det fremgår ikke, hvor længe den yderligt reducerede forsyning af gas vil stå på.Den tyske regering anser den forklaringen om vedligeholdelse for at være opfundet til lejligheden.- Ifølge vores oplysninger er der ingen teknisk grund til en reduktion i leverancerne, siger en talskvinde for Finansministeriet og minister Robert Habeck til Frankfurter Allgemeine Zeitung.Tyskerne får 25 procent af deres energi fra gas, hvor en overvejende del er kommet fra Rusland.Gasprisen stiger med 10 procentDet er anden gang indenfor en uge, at Gazprom reducerer leverancen af gas under påskud af reperation af gasturbiner. Da Gazprom efter ti dages vedligehold i sidste uge genåbnede for gasforsyningen til Tyskland, var meldingen, at der dagligt ville blive leveret cirka 67 millioner kubikmeter.Gazproms seneste melding betyder altså, at leverancerne til Europa bliver omtrent halveret fra onsdag. Gasledningen kan, når den kører for fuld kraft, levere cirka 167 millioner kubikmeter gas om dagen.Nordstream 1-faciliter i Lubmin i Tyskland. (Foto:\xa0HANNIBAL HANSCHKE ©\xa0Ritzau Scanpix)'
text

'Gazprom halverer gasleverancerne til Europa via Nord Stream 1. Årsagen er ifølge selskabet vedligehold af en gasturbine. Den daglige gasforsyning via gasledningen vil fra onsdag morgen blive reduceret til 33 millioner kubikmeter, oplyser Gazprom.Det svarer til cirka 20 procent af den maksimale kapacitet, og det fremgår ikke, hvor længe den yderligt reducerede forsyning af gas vil stå på.Den tyske regering anser den forklaringen om vedligeholdelse for at være opfundet til lejligheden.- Ifølge vores oplysninger er der ingen teknisk grund til en reduktion i leverancerne, siger en talskvinde for Finansministeriet og minister Robert Habeck til Frankfurter Allgemeine Zeitung.Tyskerne får 25 procent af deres energi fra gas, hvor en overvejende del er kommet fra Rusland.Gasprisen stiger med 10 procentDet er anden gang indenfor en uge, at Gazprom reducerer leverancen af gas under påskud af reperation af gasturbiner. Da Gazprom efter ti dages vedligehold i sidste uge genåbnede for gasforsyninge

In [50]:
import re
first_digit = re.search(r'\d+', text)  # search 找第一个
print(first_digit.group()) #group() returns the matched string
all_digits = re.findall(r'\d+', text)  # findall 找全部
all_digits

1


['1', '33', '20', '25', '10', '67', '167', '1']

In [51]:
millioner = re.findall(r'\d+ millioner', text) 
millioner

['33 millioner', '67 millioner', '167 millioner']

In [55]:
quote = re.findall(r'- [\w ]+,', text) 
quote

['- Ifølge vores oplysninger er der ingen teknisk grund til en reduktion i leverancerne,']

In [62]:
photo = re.findall(r'\(.+\)', text)
photo

['(Foto:\xa0HANNIBAL HANSCHKE ©\xa0Ritzau Scanpix)']

In [75]:
sentences = re.split(r'\.',text) #Remember that "." is a special character in regex, so we need to escape it with "\"
sentences

['Gazprom halverer gasleverancerne til Europa via Nord Stream 1',
 ' Årsagen er ifølge selskabet vedligehold af en gasturbine',
 ' Den daglige gasforsyning via gasledningen vil fra onsdag morgen blive reduceret til 33 millioner kubikmeter, oplyser Gazprom',
 'Det svarer til cirka 20 procent af den maksimale kapacitet, og det fremgår ikke, hvor længe den yderligt reducerede forsyning af gas vil stå på',
 'Den tyske regering anser den forklaringen om vedligeholdelse for at være opfundet til lejligheden',
 '- Ifølge vores oplysninger er der ingen teknisk grund til en reduktion i leverancerne, siger en talskvinde for Finansministeriet og minister Robert Habeck til Frankfurter Allgemeine Zeitung',
 'Tyskerne får 25 procent af deres energi fra gas, hvor en overvejende del er kommet fra Rusland',
 'Gasprisen stiger med 10 procentDet er anden gang indenfor en uge, at Gazprom reducerer leverancen af gas under påskud af reperation af gasturbiner',
 ' Da Gazprom efter ti dages vedligehold i sidst